In [18]:
import pandas as pd

In [19]:
df_users = pd.read_parquet('../datasets/processed/google/reviews_google.parquet.gz')

In [20]:
users = df_users['user_id'].unique()

In [21]:
df_users.head(1)

,user_id,name,time,rating,gmap_id,resp_date,state,sentiment_textblob_resp,sentiment
0,1.089912e+20,Song Ro,2021-01-06 05:12:07.056,5,0x80c2c778e3b73d33:0xbdc58662a4a97d49,NaT,California,0.0,2


In [16]:
df_users.drop(columns=['gmap_id', 'state', 'sentiment_textblob_resp', 'sentiment'], inplace=True)

In [17]:
usersG[['user_id','name']].drop_duplicates()

,user_id,name
0,1.089912e+20,Song Ro
1,1.112903e+20,Rafa Robles
2,1.126404e+20,David Han
3,1.174403e+20,Anthony Kim
4,1.005808e+20,Mario Marzouk
...,...,...
9599884,1.167003e+20,Alpha Imperial
9599887,1.077358e+20,Mary Judd Nunez
9599889,1.104871e+20,malu norada
9599932,1.166060e+20,Gregory Jimenez


In [ ]:
# Sacar milesimas de segundo
usersG['time'] = usersG['time'].astype(str)
usersG['time'] = usersG['time'].apply(lambda x: x.split('.')[0])
usersG['time'] = pd.to_datetime(usersG['time'])
usersG